In [50]:
%%capture
!pip install ipython-autotime
%load_ext autotime

import cv2
import numpy as np
from google_drive_downloader import GoogleDriveDownloader as gdd
from keras import models

time: 2.77 s (started: 2021-05-04 19:46:34 +00:00)


In [51]:
!rm -rf sample_data
filename = 'video.mp4'
gdd.download_file_from_google_drive(file_id='102TV4bTMQlU8jAJGajgKFWVoGZqBjE2M',
                                    dest_path=f'./{filename}')

time: 145 ms (started: 2021-05-04 19:46:37 +00:00)


In [52]:
video = cv2.VideoCapture(filename)
frameSize = (int(video.get(3)),int(video.get(4)))
fps = video.get(cv2.CAP_PROP_FPS)
frames = []
data = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    frames.append(np.array(frame))
    data.append(cv2.resize(frame, (256,256))/255.0)
video.release()
print(f'frames: {len(frames)}')

frames: 357
time: 2.23 s (started: 2021-05-04 19:46:37 +00:00)


In [55]:
model = models.load_model('trackerDeep.h5')
preds = model.predict(np.array(data))

time: 5.53 s (started: 2021-05-04 19:47:08 +00:00)


In [54]:
fourcc = cv2.VideoWriter.fourcc(*"MJPG")
out = cv2.VideoWriter('result.mp4', fourcc, fps, frameSize)
for i, frame in enumerate(frames):
  left = np.rint(preds[i][[0,1]] * frameSize).astype(int)
  right = np.rint(preds[i][[1,2]] * frameSize).astype(int)
  frame = cv2.circle(frame, tuple(left), radius=10, color=(0, 180, 255), thickness=-1)
  frame = cv2.circle(frame, tuple(right), radius=10, color=(255, 180, 0), thickness=-1)
  out.write(frame)
out.release()

time: 1.95 s (started: 2021-05-04 19:46:45 +00:00)
